# Model training and Evaluation

In [3]:
import os
import sys
import glob
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import seaborn as sns

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"  # Hide warning logs from tensorflow
tf.get_logger().setLevel("ERROR")

In [5]:
# Constants
IMG_SIZE = 224
BATCH_SIZE = 300
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 5
INPUT_SHAPE=(224, 224, 3)

tf.random.set_seed(5)
dataset_dir = "../datasets"

# Change dataset_dir when run in google colab 
if 'google.colab' in sys.modules:
    from google.colab import drive

    drive.mount('/content/drive')
    dataset_dir = "/content/drive/Othercomputers/Big Mac/datasets"
    BATCH_SIZE = 430

physical_gpus = tf.config.list_physical_devices('GPU')
print("Using available GPUs: ", physical_gpus)

tf.keras.mixed_precision.set_global_policy('float32')

Using available GPUs:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
# Load ImageNet2012 dataset
def prepare_input_data(input):
    image = tf.cast(input['image'], tf.float32)
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    label = input['label']
    return image, label

def make_dataset(ds):
    return (
        ds.map(prepare_input_data, num_parallel_calls=AUTOTUNE)
        .batch(BATCH_SIZE)
        .prefetch(AUTOTUNE)
    )


(train, validation, test), info = tfds.load(
    'imagenet2012_subset/10pct',
    split=['train', 'validation[:50%]', 'validation[50%:]'],
    shuffle_files=False,
    with_info=True,
    data_dir=dataset_dir
)

num_classes = info.features['label'].num_classes
class_names = info.features['label'].names

print(f"Train count: {info.splits['train'].num_examples}")
print(f"Validation count: {info.splits['validation[:50%]'].num_examples}")
print(f"Test count: {info.splits['validation[50%:]'].num_examples}")

train_dataset = make_dataset(train)
validation_dataset = make_dataset(validation)
test_dataset = make_dataset(test)

Train count: 128116
Validation[:50%] count: 25000
Validation[50%:] (test) count: 25000


In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

AUTOTUNE = tf.data.AUTOTUNE

def class_counts_from_raw_ds(raw_ds, num_classes):
    # Map to one-hot labels and sum across the dataset
    counts = (
    raw_ds
    .map(lambda x: tf.one_hot(x['label'], num_classes, dtype=tf.int64),
        num_parallel_calls=AUTOTUNE)
    .batch(4096)
    .reduce(
        initial_state=tf.zeros([num_classes], dtype=tf.int64),
        reduce_func=lambda acc, x: acc + tf.reduce_sum(x, axis=0)
    )
    )
    return counts.numpy()

def print_distribution(name, counts, class_names=None, top_k=2):
    total = counts.sum()
    print(f"\n{name}: total={total}, classes={len(counts)}")
    if class_names is None:
        class_names = [str(i) for i in range(len(counts))]
    # Show a quick summary: most/least frequent classes
    idx_sorted = np.argsort(counts)
    print(f"Least frequent {top_k}:")
    for i in idx_sorted[:top_k]:
        print(f"{i:4d} {class_names[i]:30s} {int(counts[i]):7d} ({counts[i]/total:.2%})")
    print(f"Most frequent {top_k}:")
    for i in idx_sorted[-top_k:][::-1]:
        print(f"{i:4d} {class_names[i]:30s} {int(counts[i]):7d} ({counts[i]/total:.2%})")


train_counts = class_counts_from_raw_ds(train, num_classes)
val_counts = class_counts_from_raw_ds(validation, num_classes)
test_counts = class_counts_from_raw_ds(test, num_classes)

print_distribution("Train", train_counts, class_names)
print_distribution("Validation", val_counts, class_names)
print_distribution("Test", test_counts, class_names)


Train: total=128116, classes=1000
Least frequent 2:
   0 n01440764                          128 (0.10%)
 638 n03710637                          128 (0.10%)
Most frequent 2:
 618 n03633091                          129 (0.10%)
 420 n02787622                          129 (0.10%)

Validation[:50%]: total=25000, classes=1000
Least frequent 2:
 599 n03530642                           14 (0.06%)
 901 n04579145                           14 (0.06%)
Most frequent 2:
 414 n02769748                           35 (0.14%)
 238 n02107574                           35 (0.14%)

Validation[50%:] (Test): total=25000, classes=1000
Least frequent 2:
 147 n02066245                           15 (0.06%)
 238 n02107574                           15 (0.06%)
Most frequent 2:
 901 n04579145                           36 (0.14%)
 599 n03530642                           36 (0.14%)


In [21]:
# Load adversarial datasets

def _parse_function(proto):
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(proto, feature_description)
    image_f16 = tf.io.parse_tensor(parsed_features['image'], out_type=tf.float16)
    label = parsed_features['label']
    image_f32 = tf.cast(image_f16, tf.float32)
    image_f32.set_shape([IMG_SIZE, IMG_SIZE, 3])
    return image_f32, label

def create_tf_dataset(file_paths):
    raw_dataset = tf.data.TFRecordDataset(file_paths, compression_type='GZIP')
    tf_dataset = raw_dataset.map(_parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return tf_dataset

# Get all adversarial datasets for train, validation, and testing
test_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/test-*.tfrec')
train_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/train-*.tfrec')
validation_file_paths = glob.glob(f'{dataset_dir}/adversaries/imagenet2012_subset/validation-*.tfrec')

print(f"Loaded {len(train_file_paths)} TFrecord train files")
print(f"Loaded {len(test_file_paths)} TFrecord test files")
print(f"Loaded {len(validation_file_paths)} TFrecord validation files")

# Create a TFRecordDataset
adv_test_dataset = create_tf_dataset(test_file_paths)
adv_train_dataset = create_tf_dataset(train_file_paths)
adv_validation_dataset = create_tf_dataset(validation_file_paths)

Loaded 60 TFrecord train files
Loaded 0 TFrecord test files
Loaded 55 TFrecord validation files


In [22]:
print("Training robust ResNet-50 model...\n")

base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=INPUT_SHAPE,
    classes=1000
)

print("Freezing the ResNet50 backbone...")

base_model.trainable = False

# Build a small classification head on top of the backbone
inputs = tf.keras.Input(shape=INPUT_SHAPE)
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

robust_model = tf.keras.Model(inputs, outputs)

robust_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=[
        'accuracy',
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
    ]
)

# Add validation_data and store the output in a 'history' object

robust_model.fit(
    adv_train_dataset,
    verbose=1,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=adv_validation_dataset
)

# 5. Unfreeze the backbone (or part of it) for fine-tuning. 🚀
print("Unfreezing top layers of the backbone for fine-tuning...")

base_model.trainable = True

# Unfreeze last few layers of base model
# for layer in base_model.layers[-10:]:
# for layer in base_model.layers[:-50]:

for layer in base_model.layers[-10:]:
    layer.trainable = False

print("Fine tuning with lower learning rate")

# Lower learning rate for fine-tuning

# robust_model.compile(
#     optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
#     metrics=[
#         'accuracy',
#         tf.keras.metrics.SparseTopKCategoricalAccuracy(k=5, name='top_5_accuracy'),
#     ]
# )

# robust_model.fit(
#     adv_train_dataset,
#     verbose=1,
#     batch_size=BATCH_SIZE,
#     epochs=EPOCHS,
#     validation_data=adv_validation_dataset
# )

robust_model.save("robust_resnet50.keras")


Training robust ResNet-50 model...

Freezing the ResNet50 backbone...
Epoch 1/5
     86/Unknown 60s 656ms/step - accuracy: 0.0015 - loss: 0.0316 - top_5_accuracy: 0.0060

/Users/stanleyokwii/Documents/Birkbeck/Msc Project/msc-project-source-code-files-24-25-Stanley-Okwii/.venv/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


86/86 ━━━━━━━━━━━━━━━━━━━━ 114s 1s/step - accuracy: 0.0015 - loss: 0.0274 - top_5_accuracy: 0.0058 - val_accuracy: 0.0022 - val_loss: 0.0248 - val_top_5_accuracy: 0.0065
Epoch 2/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.0016 - loss: 0.1458 - top_5_accuracy: 0.0058 - val_accuracy: 0.0010 - val_loss: 0.3479 - val_top_5_accuracy: 0.0054
Epoch 3/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 0.0011 - loss: 0.3752 - top_5_accuracy: 0.0057 - val_accuracy: 8.8795e-04 - val_loss: 0.6491 - val_top_5_accuracy: 0.0050
Epoch 4/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 103s 1s/step - accuracy: 9.6899e-04 - loss: 0.5979 - top_5_accuracy: 0.0050 - val_accuracy: 0.0011 - val_loss: 0.8293 - val_top_5_accuracy: 0.0069
Epoch 5/5
86/86 ━━━━━━━━━━━━━━━━━━━━ 106s 1s/step - accuracy: 0.0014 - loss: 0.8547 - top_5_accuracy: 0.0065 - val_accuracy: 0.0010 - val_loss: 1.5811 - val_top_5_accuracy: 0.0092
Unfreezing top layers of the backbone for fine-tuning...
Fine tuning with lower learning rate
